In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoModelForCausalLM, AutoTokenizer
from exp1 import load_jsonl
import os
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn.functional as F
from torch import nn


c:\Users\jindd\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\jindd\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
def evaluate_priming_effect(model, tokenizer, primer, index, device, k=5):
    model.eval()
    with torch.no_grad():
        # Encode the input text and send to the specified device
        encoded_input = tokenizer(primer, return_tensors='pt').to(device)
        outputs = model(**encoded_input)
        logits = outputs.logits

        # Check if the desired index is within the length of the sequence
        if index < logits.size(1):
            logits_at_index = logits[0, index, :]
            probs_at_index = F.softmax(logits_at_index, dim=0)
            top_probs, top_indices = torch.topk(probs_at_index, k)

            predictions = {}
            for idx, prob in zip(top_indices, top_probs):
                token = tokenizer.decode([idx.item()])
                predictions[token] = prob.item()
            
            return predictions
        else:
            return {"error": "Index out of bounds"}
            
def evaluate_priming_effect_at_index(sentence,index,k=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model_name = 'gpt2'
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.to( device )
    predictions = evaluate_priming_effect(model, tokenizer, sentence, index, device, k)
    return predictions
    

In [32]:

set_of_sentences = {
    "Teresa is attacked by some _ "
}

for sentence in set_of_sentences:
    # Evaluate the priming effect and get the predictions
    index = sentence.split().index("_")-1
    print( index)
    predictions = evaluate_priming_effect_at_index(sentence, index, 100)
    # Create a new file with a unique name based on the sentence
    filename = f"result_{sentence.replace(' ', '_')}.txt"
    
    # Write the predictions to the file
    with open(filename, 'w') as file:
        for token, prob in predictions.items():
            file.write(f"{token}: {prob}\n")

# evaluate_priming_effect_at_index( "The cat sat on the mat", 4, 200)


4
Using device: cuda


In [ ]:
# if __name__ == "__main__":
#     sentence = "This is a crazy world."
#     index = 2     
#     predictions = evaluate_priming_effect_at_index(sentence, index,k=5)
#     print("Predictions:", predictions)